In [1]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelBinarizer
import cv2
!pip install imutils
import imutils

  Preparing metadata (setup.py) ... - done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=4163c21731a209e16d44818d6bc1ecd4d68340a4847c1615ce4c1667829efc90
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


In [2]:
model = load_model('/kaggle/input/smartathon-training-vgg19/detector.h5')
lb = pickle.loads(open('/kaggle/input/smartathon-training-vgg19/lb.pickle', "rb").read())

2023-01-17 09:08:57.312669: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
test = pd.read_csv('/kaggle/input/smartathon-dataset-zip/dataset/test.csv')
test['image_path'] = "/kaggle/input/smartathon-dataset-zip/dataset/images/" + test['image_path']
test['image_path'] = test['image_path'].apply(lambda x: str(x))
test.head()

,image_path
0,/kaggle/input/smartathon-dataset-zip/dataset/i...
1,/kaggle/input/smartathon-dataset-zip/dataset/i...
2,/kaggle/input/smartathon-dataset-zip/dataset/i...
3,/kaggle/input/smartathon-dataset-zip/dataset/i...
4,/kaggle/input/smartathon-dataset-zip/dataset/i...


In [4]:
train = pd.read_csv('/kaggle/input/smartathon-dataset-zip/dataset/train.csv')

In [5]:
classes = list(train['class'].unique())
cn = {}
for cl in classes:
    name = train.loc[train['class']==cl, 'name'].values[0]
    cn[cl] = name

In [6]:
summary = []
for i in range(test.shape[0]):
    path = test.iloc[i, 0]
    image = load_img(test.iloc[i, 0], target_size=(224, 224))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    
    (boxPreds, labelPreds) = model.predict(image)
    (startX, startY, endX, endY) = boxPreds[0]
    
    i = np.argmax(labelPreds, axis=1)
    label = lb.classes_[i][0]
    
    image = cv2.imread(test.iloc[i, 0].values[0])
    image = imutils.resize(image, width=9120)
    (h, w) = image.shape[:2]
    
    startX = int(startX * w)
    startY = int(startY * h)
    endX = int(endX * w)
    endY = int(endY * h)
    
    name = cn[float(label)]
    
    summary.append({
        'class' : float(label),
        'imagePath' : test.iloc[i, 0],
        'name' : name,
        'xmax' : endX,
        'xmin' : startX,
        'ymax' : endY,
        'ymin' : startY
    })

2023-01-17 09:09:00.703150: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [7]:
summary = pd.DataFrame(summary)
summary.to_csv('vgg19.csv')